# yFinance
Links for References used for this excercise: 
1. https://github.com/ranaroussi/yfinance
2. https://www.geeksforgeeks.org/python/multithreaded-download-of-yahoo-stock-history-with-python-yfinance/
3. for handling dates and time: https://docs.python.org/3/library/datetime.html

In [12]:
# yahoo finance data. 
import yfinance as yf
import time
import datetime

start= time.time()
today = datetime.date.today()
fivedaysago = today - datetime.timedelta(days=5)

print (today)
print (fivedaysago)

ticker_list =['META','AAPL','AMZN','NFLX','GOOG']

data =yf.download( 
    tickers = ticker_list,
    threads=True,
    group_by='ticker',
)

data =data.T 

for t in ticker_list: 
    print(t)   
    print(data.loc[t])

# This segment allows for monitoring the time taken to complete the execution of the program. 
print ('The program takes ',time.time()-start,'seconds.')

/tmp/ipykernel_2443/3134025215.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data =yf.download(
[**********************60%****                   ]  3 of 5 completed

2025-10-08
2025-10-03


[*********************100%***********************]  5 of 5 completed

META
Date      2025-09-08    2025-09-09    2025-09-10    2025-09-11    2025-09-12  \
Price                                                                          
Open    7.554901e+02  7.569790e+02  7.646139e+02  7.541410e+02  7.482250e+02   
High    7.659930e+02  7.657831e+02  7.651836e+02  7.565893e+02  7.570590e+02   
Low     7.515128e+02  7.529218e+02  7.504935e+02  7.478652e+02  7.432583e+02   
Close   7.517925e+02  7.651835e+02  7.514728e+02  7.503936e+02  7.550804e+02   
Volume  1.308780e+07  1.099900e+07  1.247830e+07  7.923300e+06  8.248600e+06   

Date      2025-09-15    2025-09-16    2025-09-17    2025-09-18    2025-09-19  \
Price                                                                          
Open    7.569590e+02  7.664826e+02  7.794639e+02  7.802234e+02  7.858895e+02   
High    7.735479e+02  7.808329e+02  7.827617e+02  7.882480e+02  7.902666e+02   
Low     7.514828e+02  7.645839e+02  7.657932e+02  7.728384e+02  7.686712e+02   
Close   7.641842e+02  7.784745e+02

## End